# Sex differences in Autism Spectrum Disorder, a Comorbidity Pattern Analysis in National Scale Data: (i)data selection

## Load the R libraries

In [ ]:
library("devtools")
library("SqlServerJtds")
library("SqlTools")
library("FactToCube")

## Set up the connection the the DB server

In [ ]:
cn = connect.sql.server(
    database="XXXX",
    domain="YYYY",
    server.address="ZZZZ",
    user= yourUserName,
    password= yourPassword)
cn


## Mapping table: PheWAS to ICD9-CM code
We create a data.frame with ICD9, the equivalent PheCode based on (https://phewascatalog.org/, version 1.2)and if it is sex specific or not (https://phewascatalog.org/files/). Once the data.frame has been created we save this data.frame as a table in our sql database. This will allow us to map and to remove potential patient with diagnostic errors due to sex specific phenotypes.  

In [ ]:
icd9tophewas <- read.delim("phecode_icd9_rolled.csv", 
                           colClasses = "character",
                           sep=",", 
                           header=TRUE)

icd9tophewas <- icd9tophewas[, c(1:4)]
colnames(icd9tophewas)[2] <- "ICD9description"

icd9sexrelated <- read.delim("phecode_definitions1.2.csv", 
                           colClasses = "character",
                           sep=",", 
                           header=TRUE)
icd9sexrelated <- icd9sexrelated[,c("jd_code", "sex")]
colnames(icd9sexrelated) <- c("PheCode", "Sex")

mymappingtable <- merge( icd9tophewas, icd9sexrelated, by="PheCode")

dbWriteTable( cn, "icd9tophewas", mymappingtable, row.names = FALSE)

## ASD patient selection 

Create a temporal table with the Autism Spectrum Disorder ICD9-CM codes.

In [ ]:
dbGetQuery(cn,  "
           SELECT
            * INTO #tmpAsdIcdCodes
           FROM
           PheWAS.dbo.Icd9CodeTranslation F WHERE
             F.Icd9Code = '299.0' OR F.Icd9Code  = '299.00' OR F.Icd9Code  = '299.01' OR F.Icd9Code  = '299.8' OR 
             F.Icd9Code  = '299.80' OR F.Icd9Code  = '299.81' OR F.Icd9Code  = '299.9' OR F.Icd9Code  = '299.90' OR
             F.Icd9Code  = '299.91'")

Select those patients diagnosed with ASD from 0 to 18 years old.

In [ ]:
dbSendUpdate(cn, "
             SELECT F.MemberId, F.Icd, F.DateServiceStarted, M.Gender, M.BirthYear, 
             (YEAR( F.DateServiceStarted ) - M.BirthYear) AS Age
             INTO #tmpAsdIcd
             FROM FactIcd F
             INNER JOIN Members M ON
             F.MemberId=M.MemberId
             WHERE
             F.Icd IN (SELECT Icd9Code FROM #tmpAsdIcdCodes)
             AND 
             (YEAR( F.DateServiceStarted ) - M.BirthYear) <= 18
             AND 
             (YEAR( F.DateServiceStarted ) - M.BirthYear) >= 0
             AND IcdVersion = '9'" 
)

Select only those patients that have been diagnosed at least 3 times with a ASD ICD9-CM code.

In [ ]:
dbSendUpdate(cn, "
             SELECT
             T.MemberId, COUNT (T.Icd ) AS icdCounts
             INTO #AsdIcdCounts
             FROM
             #tmpAsdIcd T
             GROUP BY MemberId 
             HAVING COUNT (T.Icd ) >= 3" 
)

From those, select those with a coverage of 12 months between 0 to 18 years old, and only those from where we also know gender for sure.

In [ ]:
dbSendUpdate(cn, "
             SELECT
             E.MemberId, COUNT(DISTINCT E.EffectiveDate) AS nMonthsObservation
             INTO ag440.dbo.ASD3_count
             FROM
             Enrollment E
             INNER JOIN Members M ON
             E.MemberId=M.MemberId
             INNER JOIN #AsdIcdCounts C ON
             M.MemberId=C.MemberId
             WHERE
             E.MedicalIndicator = 'Y'
             AND (YEAR(E.EffectiveDate) - M.BirthYear) <= 18
             AND (YEAR(E.EffectiveDate) - M.BirthYear) >= 0
             AND (M.Gender = 'F' OR M.Gender = 'M') 
             GROUP BY E.MemberId
             HAVING COUNT( DISTINCT E.EffectiveDate) >=12 " 
)

We generate two different and separate tables, one with the females and the other with males. 

In [ ]:
#females, ASD 3 times, from 0 to 18 years old, and coverage of at least 12 months
dbSendUpdate(cn, "
             SELECT
             E.MemberId, COUNT(DISTINCT E.EffectiveDate) AS nMonthsObservation
             INTO ag440.dbo.ASDfemales_count
             FROM
             Enrollment E
             INNER JOIN Members M ON
             E.MemberId=M.MemberId
             INNER JOIN #AsdIcdCounts C ON
             M.MemberId=C.MemberId
             WHERE
             E.MedicalIndicator = 'Y'
             AND (YEAR(E.EffectiveDate) - M.BirthYear) <= 18
             AND (YEAR(E.EffectiveDate) - M.BirthYear) >= 0
             AND (M.Gender = 'F') 
             GROUP BY E.MemberId
             HAVING COUNT( DISTINCT E.EffectiveDate) >=12 " 
)

#males, ASD 3 times, from 0 to 18 years old, and coverage of at least 12 months
dbSendUpdate(cn, "
             SELECT
             E.MemberId, COUNT(DISTINCT E.EffectiveDate) AS nMonthsObservation
             INTO ag440.dbo.ASDmales_count
             FROM
             Enrollment E
             INNER JOIN Members M ON
             E.MemberId=M.MemberId
             INNER JOIN #AsdIcdCounts C ON
             M.MemberId=C.MemberId
             WHERE
             E.MedicalIndicator = 'Y'
             AND (YEAR(E.EffectiveDate) - M.BirthYear) <= 18
             AND (YEAR(E.EffectiveDate) - M.BirthYear) >= 0
             AND (M.Gender = 'M') 
             GROUP BY E.MemberId
             HAVING COUNT( DISTINCT E.EffectiveDate) >=12 " 
)

## Control patient data selection

First we need a table with all patients in the db with 1 ASD code for exclusion (we already create it, "tmpAsdIcd").
Then we generate a table with all the patients with 12 months of coverage between 0 to 18 years old

In [ ]:
dbSendUpdate(cn, "SELECT
             E.MemberId, datediff(month, min(E.EffectiveDate), max(E.EffectiveDate)) AS months
             INTO #tmpMemberCount
             FROM
             Enrollment E
             INNER JOIN Members M ON
             E.MemberId=M.MemberId
             WHERE
             E.MedicalIndicator = 'Y'
             AND (YEAR(E.EffectiveDate) - M.BirthYear) <= 18
             AND (YEAR(E.EffectiveDate) - M.BirthYear) >= 0
             AND (M.Gender = 'M' OR M.Gender = 'F') 
             GROUP BY E.MemberId
             HAVING  COUNT ( DISTINCT E.EffectiveDate ) > 12"
)

We remove those patients diagnosed with ASD at some point

In [ ]:
dbSendUpdate( cn, "SELECT * 
                   INTO #tmNonASDMemberCount 
                   FROM #tmpMemberCount 
                   WHERE MemberId NOT IN (SELECT MemberId FROM #tmpAsdIcd)" 
)

For those patients, we sleect the id and the gender and save it on a table.
We then generate two separate tables, one for females and another one for males. 

In [ ]:
dbSendUpdate( cn, "SELECT M.MemberId, M.Gender, E.ZipCode
                    INTO #allNonASD
                    FROM Members M
                    INNER JOIN #tmNonASDMemberCount T ON
                    M.MemberId=T.MemberId
                    INNER JOIN Enrollment E ON
                    E.MemberId=M.MemberId" 
)

#Create non-ASD female control group filter
dbSendUpdate( cn, "SELECT MemberId 
                    INTO ag440.dbo.nonASDfemales_count 
                    FROM #allNonASD 
                    WHERE Gender = 'F'"
)

#Create non-ASD male control group filter
dbSendUpdate( cn, "SELECT MemberId 
                    INTO ag440.dbo.nonASDmales_count 
                    FROM #allNonASD 
                    WHERE Gender = 'M'"
)

## Prepare data for the analysis

### ASD females from 0 to 2 years old

In [ ]:
dbSendUpdate( cn, "SELECT
             F.MemberId, F.Icd,
             M.BirthYear,F.DateServiceStarted,
             YEAR(F.DateServiceStarted) - M.BirthYear AS Age
             INTO #tmpAsdFemIcd
             FROM
             FactIcd F
             INNER JOIN Members M ON
             F.MemberId=M.MemberId
             INNER JOIN ag440.dbo.ASDfemales_count T ON
             M.MemberId=T.MemberId
             WHERE
             IcdVersion = '9'")

Remove patients with diagnostic errores, females diagnosed with ICD9-CM codes that are male specific. 

In [ ]:
dbSendUpdate( cn, "SELECT MemberId 
                    INTO #tmptest 
                    FROM #tmpAsdFemIcd 
                    WHERE Icd 
                    IN (SELECT ICD9 
                        FROM ag440.dbo.icd9tophewas
                        WHERE Sex = 'Male') "
)

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.ASDfemalesICD 
                    FROM #tmpAsdFemIcd 
                    WHERE MemberId 
                    NOT IN (SELECT DISTINCT( MemberId )  
                            FROM #tmptest)
")

Filter by age, for example, diagnostic done from 0 to 2 years old. 

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO #tmpFemalesASD0to2 
                    FROM #tmpAsdFemIcdNoError 
                    WHERE Age >= 0 AND Age <= 2 
")

Map the ICD9-CM codes to PheWAS code

In [ ]:
dbSendUpdate( cn, "SELECT A.MemberId, A.BirthYear, A.DateServiceStarted, A.Age, A.Icd, B.PheCode
                    INTO #tmpFemalesASD0to2PheWAS
                    FROM #tmpFemalesASD0to2 A 
                    INNER JOIN ag440.dbo.icd9tophewas B ON
                    A.Icd = B.ICD9"
)

Add the counts

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId, T.PheCode, COUNT(T.PheCode) AS Count
                    INTO #phewasCountFemalesASD0to2
                    FROM #tmpFemalesASD0to2PheWAS T
                    GROUP BY T.MemberId, T.PheCode"
)

Join the two tables

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheCode, P.Count
                    INTO #finalASDfem0to2
                    FROM #tmpFemalesASD0to2PheWAS T
                    INNER JOIN #phewasCountFemalesASD0to2 P 
                    ON T.MemberId = P.MemberId AND T.PheCode = P.PheCode"
)

Remove those ICD9-CM codes that are autism codes

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.ASDfem0to2Comorbidities 
                    FROM #finalASDfem0to2 F 
                    WHERE F.Icd 
                    NOT IN (SELECT I.Icd9Code 
                            FROM #tmpAsdIcdCodes I)"
)

Remove duplicate rows (if any)

In [ ]:
dbSendUpdate( cn, "WITH CTE AS(
             SELECT *, ROW_NUMBER() 
             OVER ( PARTITION BY MemberId, BirthYear, DateServiceStarted, Age, Icd,  
             PheCode, Count ORDER BY MemberId, BirthYear, DateServiceStarted, Age, Icd,  
             PheCode, Count) AS RN
             FROM ag440.dbo.ASDfem0to2Comorbidities)
             DELETE FROM CTE WHERE RN<>1"
)

### ASD males from 0 to 2 years old

In [ ]:
dbSendUpdate( cn, "SELECT
             F.MemberId, F.Icd,
             M.BirthYear,F.DateServiceStarted,
             YEAR(F.DateServiceStarted) - M.BirthYear AS Age
             INTO #tmpAsdMaleIcd
             FROM
             FactIcd F
             INNER JOIN Members M ON
             F.MemberId=M.MemberId
             INNER JOIN ag440.dbo.ASDmales_count T ON
             M.MemberId=T.MemberId
             WHERE
             IcdVersion = '9'"
)

Remove patients with diagnostic errores, females diagnosed with ICD9-CM codes that are male specific. 

In [ ]:
dbSendUpdate( cn, "SELECT MemberId 
                    INTO #tmptestMale 
                    FROM #tmpAsdMaleIcd 
                    WHERE Icd 
                    IN (SELECT ICD9 
                    FROM ag440.dbo.icd9tophewas 
                    WHERE Sex = 'Female') "
)

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.ASDmalesICD 
                    FROM #tmpAsdMaleIcd 
                    WHERE MemberId 
                    NOT IN ( SELECT DISTINCT( MemberId )  
                    FROM #tmptestMale)"
)

Filter by age, for example, diagnostic done from 0 to 2 years old. 

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO #tmpMalesASD0to2 
                    FROM #tmpAsdMaleIcdNoError 
                    WHERE Age >= 0 AND Age <= 2"
)

Map the ICD9-CM codes to PheWAS code

In [ ]:
dbSendUpdate( cn, "SELECT A.MemberId, A.BirthYear, A.DateServiceStarted, A.Age, A.Icd, B.PheCode
                    INTO #tmpMalesASD0to2PheWAS
                    FROM #tmpMalesASD0to2 A 
                    INNER JOIN ag440.dbo.icd9tophewas B ON
                    A.Icd = B.ICD9"
)

Add the counts

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId, T.PheCode, COUNT(T.PheCode) AS Count
                    INTO #phewasCountMalesASD0to2
                    FROM #tmpMalesASD0to2PheWAS T
                    GROUP BY T.MemberId, T.PheCode"
)

Join the two tables

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheCode, P.Count
                    INTO #finalASDmale0to2
                    FROM #tmpMalesASD0to2PheWAS T
                    INNER JOIN #phewasCountMalesASD0to2 P ON
                    T.MemberId = P.MemberId AND T.PheCode = P.PheCode"
)

Remove those ICD9-CM codes that are autism codes

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.ASDmales0to2Comorbidities 
                    FROM #finalASDmale0to2 F 
                    WHERE F.Icd 
                    NOT IN ( SELECT I.Icd9Code 
                    FROM #tmpAsdIcdCodes I)"
)

Remove duplicate rows (if any)

In [ ]:
dbSendUpdate( cn, "WITH CTE AS(
             SELECT *, ROW_NUMBER() 
             OVER ( PARTITION BY MemberId, BirthYear, DateServiceStarted, Age, Icd,  
             PheCode, Count ORDER BY MemberId, BirthYear, DateServiceStarted, Age, Icd,  
             PheCode, Count) AS RN
             FROM
             ag440.dbo.ASDmales0to2Comorbidities
             )
             DELETE FROM CTE WHERE RN<>1")

### Females without ASD from 0 to 2 years old

For the previous generated table containing all the identifiers for females without ASD, we extract the ICD9-CM codes as well as the birth year and the date service started, and the age estimated with both variables, that will allow us to filter by age of diagnostic. 

In [ ]:
dbSendUpdate( cn, "SELECT F.MemberId, F.Icd, M.BirthYear,F.DateServiceStarted,
                     YEAR(F.DateServiceStarted) - M.BirthYear AS Age
                     INTO ag440.dbo.nonASDfemalesIcdCode
                     FROM FactIcd F
                     INNER JOIN Members M ON
                     F.MemberId=M.MemberId
                     INNER JOIN ag440.dbo.nonASDfemales_count T ON
                     M.MemberId=T.MemberId
                     WHERE IcdVersion = '9'
                     GROUP BY F.MemberId, F.Icd, M.BirthYear,F.DateServiceStarted"
)

Then to detect the females with potential diagnostic errors, we create two different tables, one with the whole list of females without ASD and another list with the ones with error diagnostic. Then we use both lists to create the one without errors. 

In [ ]:
#all females without ASD
dbSendUpdate( cn, "SELECT DISTINCT( MemberId ) 
                    INTO #tmpAllNonASDfemales 
                    FROM  ag440.dbo.nonASDfemalesIcdCode "
)

#all females without ASD with at least one diagnostic error
#(male specific disease assigned to a female)
dbSendUpdate( cn, "SELECT MemberId 
                    INTO #errorFemales 
                    FROM ag440.dbo.nonASDfemalesIcdCode 
                    WHERE Icd IN (SELECT ICD9 FROM ag440.dbo.icd9tophewas 
                    WHERE Sex = 'Male') "
)

#list with the females without errors
dbSendUpdate( cn, "SELECT * 
                    INTO #femalesNoError 
                    FROM #tmpAllNonASDfemales 
                    WHERE MemberId 
                    NOT IN ( SELECT DISTINCT( MemberId ) 
                    FROM #errorFemales)"
)

#subset of our initial table with the ICD information
#for the females without ASD and without errors
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.nonASDfemalesICD 
                    FROM ag440.dbo.nonASDfemalesIcdCode 
                    WHERE MemberId 
                    IN ( SELECT MemberId 
                    FROM #femalesNoError )"
)

Filter by age, for example, diagnostic done from 0 to 2 years old. 

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO #tmpFemalesASD0to2 
                    FROM ag440.dbo.nonASDfemalesICD 
                    WHERE Age >= 0 AND Age <= 2 "
)

Map the ICD9-CM codes to PheWAS code

In [ ]:
dbSendUpdate( cn, "SELECT A.MemberId, A.BirthYear, A.DateServiceStarted, A.Age, A.Icd, B.PheWASCode
                    INTO #tmpFemalesASD0to2PheWAS
                    FROM #tmpFemalesASD0to2 A 
                    INNER JOIN PheWAS.dbo.Icd9CodeTranslation B ON
                    A.Icd = B.Icd9Code"
)

Add the counts

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId, T.PheWAScode, COUNT(T.PheWASCode) AS Count
                    INTO #phewasCountFemalesASD0to2
                    FROM #tmpFemalesASD0to2PheWAS T
                    GROUP BY T.MemberId, T.PheWAScode"
)

Join the two tables

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheWAScode, P.Count
                    INTO ag440.dbo.nonASDfemale0to2Comorbidities
                    FROM #tmpFemalesASD0to2PheWAS T
                    INNER JOIN #phewasCountFemalesASD0to2 P ON
                    T.MemberId = P.MemberId AND T.PheWAScode = P.PheWAScode
                    GROUP BY T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheWAScode, P.Count"
)

### Males without ASD from 0 to 2 years old

Now we will do the same for males without ASD. For the previous generated table containing all the identifiers for males without ASD, we extract the ICD9-CM codes as well as the birth year and the date service started, and the age estimated with both variables, that will allow us to filter by age of diagnostic.

In [ ]:
dbSendUpdate( cn, "SELECT F.MemberId, F.Icd, M.BirthYear,F.DateServiceStarted, 
                    YEAR(F.DateServiceStarted) - M.BirthYear AS Age 
                    INTO ag440.dbo.nonASDmalesIcdCode 
                    FROM FactIcd F 
                    INNER JOIN Members M 
                    ON F.MemberId=M.MemberId 
                    INNER JOIN ag440.dbo.nonASDmales_count T
                    ON M.MemberId=T.MemberId 
                    WHERE IcdVersion = '9'
                    GROUP BY 
                    F.MemberId, F.Icd, M.BirthYear,F.DateServiceStarted"
)

Then to detect the males with potential diagnostic errors, we create two different tables, one with the whole list of males without ASD and another list with the ones with error diagnostic. Then we use both lists to create the one without errors.

In [ ]:
#all males without ASD
dbSendUpdate( cn, "SELECT DISTINCT( MemberId ) 
                    INTO #tmpAllNonASDmales 
                    FROM  ag440.dbo.nonASDmalesIcdCode"
)

#all males without ASD with at least one diagnostic error
#(female specific disease assigned to a male)
dbSendUpdate( cn, "SELECT MemberId 
                    INTO #errorMales 
                    FROM ag440.dbo.nonASDmalesIcdCode 
                    WHERE Icd IN (SELECT ICD9 
                    FROM ag440.dbo.icd9tophewas 
                    WHERE Sex = 'Female') "
)

#list with the males without errors
dbSendUpdate( cn, "SELECT * 
                    INTO #malesNoError 
                    FROM #tmpAllNonASDmales 
                    WHERE MemberId NOT IN ( SELECT DISTINCT( MemberId ) 
                    FROM #errorMales)"
)

#subset of our initial table with the ICD information
#for the males without ASD without errors
dbSendUpdate( cn, "SELECT * 
                    INTO ag440.dbo.nonASDmalesICD 
                    FROM ag440.dbo.nonASDmalesIcdCode 
                    WHERE MemberId IN ( SELECT MemberId 
                    FROM #malesNoError )
"
)

Filter by age, for example, diagnostic done from 0 to 2 years old.

In [ ]:
dbSendUpdate( cn, "SELECT * 
                    INTO #tmpMalesASD0to2 
                    FROM ag440.dbo.nonASDmalesICD 
                    WHERE Age >= 0 AND Age <= 2 "
)

Map the ICD9-CM codes to PheWAS code

In [ ]:
dbSendUpdate( cn, "SELECT A.MemberId, A.BirthYear, A.DateServiceStarted, A.Age, A.Icd, B.PheWASCode
                    INTO #tmpMalesASD0to2PheWAS
                    FROM #tmpMalesASD0to2 A 
                    INNER JOIN PheWAS.dbo.Icd9CodeTranslation B ON
                    A.Icd = B.Icd9Code"
)

Add the counts

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId, T.PheWAScode, COUNT(T.PheWASCode) AS Count
                    INTO #phewasCountMalesASD0to2
                    FROM #tmpMalesASD0to2PheWAS T
                    GROUP BY T.MemberId, T.PheWAScode"
)

Join the two tables

In [ ]:
dbSendUpdate( cn, "SELECT T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheWAScode, P.Count
                    INTO ag440.dbo.nonASDmale0to2Comorbidities
                    FROM #tmpMalesASD0to2PheWAS T
                    INNER JOIN #phewasCountMalesASD0to2 P ON
                    T.MemberId = P.MemberId AND T.PheWAScode = P.PheWAScode
                    GROUP BY T.MemberId,  T.BirthYear, T.DateServiceStarted, T.Age, T.Icd, T.PheWAScode, P.Count"
)